In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import time

In [ ]:
import torch
import numpy as np

In [ ]:
sys.path.insert(1, os.path.join("C:\\", "Soham", "Git", "spectral-NN", "source_codes"))
#sys.path.insert(1, os.path.join("C:\\", "Users", "Soham", "Git", "spectral-NN", "source_codes"))
#sys.path.insert(1, os.path.join("/home", "soham", "GitHub", "spectral-NN", "source_codes"))

import SpectralNetworks as spectNN
import Important_functions as Ifn
import spectral_NN_setup as setup

In [ ]:
dirc = "C:\\Soham\\Git\\spectral-NN\\Data\\"
#dirc = "C:\\Users\\Soham\\Git\\spectral-NN\\Data\\"
#dirc = "/home/soham/GitHub/spectral-NN/Data/"

In [ ]:
act_fn=torch.nn.Sigmoid()
init=torch.nn.init.xavier_normal_
wt_fn = Ifn.kern_truncated
#wt_fn = Ifn.kern_bartlett
#wt_fn = Ifn.kern_parzen
epochs = 1000
replicates = range(25)

In [ ]:
if not os.path.isdir("Results"):
    os.mkdir("Results")

In [ ]:
def spectral_NN_Deep(M,L,depth,width,q):
    for repl in replicates:
        print('Example'+str(repl+1)+':')
        u = np.loadtxt(dirc+"locations.dat",dtype="float32")
        if len(u.shape)==1:
            D, d = len(u), 1
            u = u.reshape(D,1)
        else:
            D, d = u.shape
        u = torch.from_numpy(u)
        x = np.loadtxt(dirc+'Example'+str(repl+1)+'.dat',dtype='float32')
        N = x.shape[0]
        if x.shape[1] != D:
            exit('Data shape mismatch!! Aborting..')
        print('N='+str(N)+', D='+str(D)+', d='+str(d))

        x = torch.from_numpy(x)
        x = x - torch.mean(x,dim=0,keepdim=True)
    
        #model = spectNN.spectralNNShallow(N,d,M,L,act_fn,init)
        model = spectNN.spectralNNDeep(N,d,M,L,depth,width,act_fn,init)
        #model = spectNN.spectralNNDeepshared1(N,d,M,L,depth,width,act_fn,init)
        #model = spectNN.spectralNNDeepshared2(N,d,M,L,depth,width,act_fn,init)
        #model = spectNN.spectralNNDeepshared3(N,d,M,L,depth,width,act_fn,init)

        err_file = os.path.join("Results","spectral_Deep_"+str(M)+"_"+str(L)+"_"+str(depth)+"_"+str(width)+".txt")

        optimizer = torch.optim.Adam(model.params,lr=0.01)
        loss = Ifn.loss_spectralNN(N, wt_fn, grid_size=100, q=q)
        print("Fitting the model ...")
        start_time = time.time()
        #l_tr = []
        for epoch in range(epochs):
            l = loss.loss_fn(x,model(u))
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            #l_tr.append(l.item())
        time_ellapsed = time.time() - start_time
        print("Time taken: {} seconds" .format(time_ellapsed))

        with torch.no_grad():
            num = loss.loss_fn(x,model(u)).item()
            den = loss.loss_fn(x,0*x).item()
            train_err = num/den
        print("Relative training error: {:.2f}%" .format(train_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))

        spect_dens_est = Ifn.spectral_density_evaluation(model, q=q, wt_fn=wt_fn)
        theta_file = dirc+"True_thetas"+str(repl+1)+".dat"
        loc_file = dirc+"True_locations"+str(repl+1)+".dat"
        spect_file = dirc+"True_spectrum"+str(repl+1)+".dat"

        test_err,num,den,tr_re,tr_im,err_re,err_im = Ifn.spectral_error_computation(spect_dens_est,theta_file,loc_file,spect_file)

        print("Relative test error: {:.2f}%" .format(test_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))
        print("Real part: Error - {:.4f}, Actual - {:.4f}" .format(err_re,tr_re))
        print("Imaginary part: Error - {:.4f}, Actual - {:.4f}" .format(err_im,tr_im))
        f_err = open(err_file,"a")
        f_err.write("Example{}:\n" .format(repl+1))
        f_err.write("Fitting time - {:.10f} seconds\n" .format(time_ellapsed))
        f_err.write("Relative errors: Training - {:.10f}\tTest - {:.10f}\n" .format(train_err,test_err))
        f_err.write("Real part: Error - {:.4f}, Actual - {:.4f}\n" .format(err_re,tr_re))
        f_err.write("Imaginary part: Error - {:.4f}, Actual - {:.4f}\n\n" .format(err_im,tr_im))
        f_err.close()
    return 0.